In [1]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3633659716073190612
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4934185779
locality {
  bus_id: 1
  links {
  }
}
incarnation: 8611373525260042280
physical_device_desc: "device: 0, name: GeForce GTX 980 Ti, pci bus id: 0000:01:00.0, compute capability: 5.2"
]


In [2]:
import sys

import tensorflow.keras
import pandas as pd
import sklearn as sk
import tensorflow as tf

print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")

Tensor Flow Version: 2.1.0
Keras Version: 2.2.4-tf

Python 3.7.7 (default, May  6 2020, 11:45:54) [MSC v.1916 64 bit (AMD64)]
Pandas 1.1.1
Scikit-Learn 0.20.3
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU is available


In [1]:
#Getting all csv files
import glob
files = sorted(glob.glob("*.csv"))
display(len(files))

319

In [2]:
import numpy as np
import pandas as pd
#epochs = np.array([]).reshape(0,136)

epochs = list()

for epoch in files:
  #print(epoch)
  dfAux = pd.read_csv(epoch,sep=',', index_col=False)
  del dfAux['TTIME']
  for col in dfAux.columns:
    dfAux[col] = dfAux[col].astype('float64')
  #epochs = np.vstack((epochs,dfAux.values))
  epochs.append(dfAux.values)
  dfAux = None

epochsArray = np.array(epochs)
"""
display(epochsArray.shape)
display(epochsArray[0])
display(epochsArray[1])
"""

C:\Users\Bryan\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


'\ndisplay(epochsArray.shape)\ndisplay(epochsArray[0])\ndisplay(epochsArray[1])\n'

In [3]:
display(epochsArray.shape)
display(epochsArray[0])
type(epochsArray[0])

(319,)

array([[247., 212., 249., ..., 283., 299., 282.],
       [247., 214., 248., ..., 281., 296., 280.],
       [245., 216., 246., ..., 281., 293., 280.],
       ...,
       [227., 194., 229., ..., 278., 309., 277.],
       [228., 192., 229., ..., 274., 307., 273.],
       [227., 191., 229., ..., 275., 306., 273.]])

numpy.ndarray

In [4]:
#Lableing the data
#0 = Lie / 1 = Truth

labels = np.array([])

for i in range(320):
  if i % 2 == 0:
    labels = np.append(labels,1)
  else:
    labels = np.append(labels,0)

#Removing the Label of video WF010_4PL since it has incomplete data
labels = np.delete(labels,187)

display(labels.shape)

(319,)

In [5]:
from sklearn.preprocessing import MinMaxScaler
def to_sequences(seq_size, obs, label):
    x = []
    y = []
    scaler = MinMaxScaler()
    obs = scaler.fit_transform(obs)
    for i in range(len(obs)-seq_size):
        window = obs[i:(i+seq_size)]
        x.append(window)
        y.append(label)
        
    return np.array(x),np.array(y)
    
SEQUENCE_SIZE = 10

xAux0, yAux0 = to_sequences(SEQUENCE_SIZE,epochsArray[0],labels[0])
xAux1, yAux1 = to_sequences(SEQUENCE_SIZE,epochsArray[1],labels[1])

X = np.vstack((xAux0,xAux1))
y = np.hstack((yAux0,yAux1))

#print(X.shape)
#print(y.shape)

In [6]:
for i in range(317):
  xAux, yAux = to_sequences(SEQUENCE_SIZE, epochsArray[i+2], labels[i+2])
  X = np.vstack((X,xAux))
  y = np.hstack((y,yAux))

print(X.shape)
print(y.shape)

(226729, 10, 136)
(226729,)


In [7]:
import math
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import LayerNormalization 
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from tensorflow.keras.utils import to_categorical
import tensorflow as tf

encoder = LabelEncoder()
encoder.fit(y)
encoded_Y = encoder.transform(y)
dummy_y = to_categorical(encoded_Y)

# Modelo 1 - Simple LSTM

In [54]:
#Modelos Oficiales 

#Modelo 1 - LSTM 
def lstm_model():
  model = Sequential()
  model.add(LSTM(64, input_shape=(X.shape[1], X.shape[2])))
  model.add(Dense(2, activation='softmax'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', 'AUC','MeanSquaredError'])
  return model

estimator_lstm = KerasClassifier(build_fn=lstm_model, epochs=12, batch_size=16) #Epochs = 12
estimator_lstm.fit(X,dummy_y)

Train on 226729 samples
Epoch 1/12
226729/226729 [==============================] - 114s 505us/sample - loss: 0.6744 - accuracy: 0.5688 - AUC: 0.6002 - MeanSquaredError: 0.2410
Epoch 2/12
226729/226729 [==============================] - 124s 545us/sample - loss: 0.6134 - accuracy: 0.6490 - AUC: 0.7138 - MeanSquaredError: 0.2140
Epoch 3/12
226729/226729 [==============================] - 125s 549us/sample - loss: 0.5517 - accuracy: 0.7029 - AUC: 0.7837 - MeanSquaredError: 0.1887
Epoch 4/12
226729/226729 [==============================] - 126s 556us/sample - loss: 0.4959 - accuracy: 0.7449 - AUC: 0.8331 - MeanSquaredError: 0.1666
Epoch 5/12
226729/226729 [==============================] - 127s 558us/sample - loss: 0.4431 - accuracy: 0.7802 - AUC: 0.8706 - MeanSquaredError: 0.1468
Epoch 6/12
226729/226729 [==============================] - 128s 563us/sample - loss: 0.4014 - accuracy: 0.8054 - AUC: 0.8960 - MeanSquaredError: 0.1314
Epoch 7/12
226729/226729 [==============================] 

# Modelo 1 - Simple LSTM (Cross Validation K = 5)

In [55]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import *

kfold = KFold(n_splits=5, shuffle=True)
results = cross_validate(estimator_lstm, X,dummy_y , cv=kfold)

#print(results.keys())
#print(results)  
#print("LSTM: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
#display(conf_mat)

Train on 181383 samples
Epoch 1/12
181383/181383 [==============================] - 91s 502us/sample - loss: 0.6782 - accuracy: 0.5624 - AUC: 0.5896 - MeanSquaredError: 0.2428
Epoch 2/12
181383/181383 [==============================] - 100s 550us/sample - loss: 0.6343 - accuracy: 0.6275 - AUC: 0.6836 - MeanSquaredError: 0.2229
Epoch 3/12
181383/181383 [==============================] - 100s 554us/sample - loss: 0.5842 - accuracy: 0.6790 - AUC: 0.7509 - MeanSquaredError: 0.2015
Epoch 4/12
181383/181383 [==============================] - 101s 554us/sample - loss: 0.5398 - accuracy: 0.7150 - AUC: 0.7964 - MeanSquaredError: 0.1835
Epoch 5/12
181383/181383 [==============================] - 98s 540us/sample - loss: 0.4962 - accuracy: 0.7455 - AUC: 0.8335 - MeanSquaredError: 0.1665
Epoch 6/12
181383/181383 [==============================] - 102s 562us/sample - loss: 0.4569 - accuracy: 0.7719 - AUC: 0.8620 - MeanSquaredError: 0.1517
Epoch 7/12
181383/181383 [==============================] - 

Epoch 12/12
181383/181383 [==============================] - 63s 346us/sample - loss: 0.2924 - accuracy: 0.8686 - AUC: 0.9474 - MeanSquaredError: 0.0920
Train on 181384 samples
Epoch 1/12
181384/181384 [==============================] - 94s 517us/sample - loss: 0.6769 - accuracy: 0.5637 - AUC: 0.5932 - MeanSquaredError: 0.2422
Epoch 2/12
181384/181384 [==============================] - 89s 493us/sample - loss: 0.6263 - accuracy: 0.6379 - AUC: 0.6962 - MeanSquaredError: 0.2194
Epoch 3/12
181384/181384 [==============================] - 89s 491us/sample - loss: 0.5742 - accuracy: 0.6863 - AUC: 0.7615 - MeanSquaredError: 0.1975
Epoch 4/12
181384/181384 [==============================] - 88s 486us/sample - loss: 0.5286 - accuracy: 0.7220 - AUC: 0.8059 - MeanSquaredError: 0.1793
Epoch 5/12
181384/181384 [==============================] - 91s 501us/sample - loss: 0.4865 - accuracy: 0.7512 - AUC: 0.8404 - MeanSquaredError: 0.1630
Epoch 6/12
181384/181384 [==============================] - 91s

In [ ]:
estimator_lstm.model.save("modelo1_LSTM.h5")

# Modelo 2 - Simple GRU

In [58]:
#Modelo 2 - GRU 
def gru_model():
  model = Sequential()
  model.add(GRU(64, input_shape=(X.shape[1], X.shape[2])))
  model.add(Dense(2, activation='softmax'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', 'AUC','MeanSquaredError'])
  return model

estimator_GRU = KerasClassifier(build_fn=gru_model, epochs=12, batch_size=16) #Epochs = 12
estimator_GRU.fit(X,dummy_y)

Train on 226729 samples
Epoch 1/12
226729/226729 [==============================] - 93s 412us/sample - loss: 0.6671 - accuracy: 0.5802 - AUC: 0.6189 - MeanSquaredError: 0.2377
Epoch 2/12
226729/226729 [==============================] - 93s 411us/sample - loss: 0.5849 - accuracy: 0.6771 - AUC: 0.7499 - MeanSquaredError: 0.2019
Epoch 3/12
226729/226729 [==============================] - 97s 426us/sample - loss: 0.5109 - accuracy: 0.7361 - AUC: 0.8217 - MeanSquaredError: 0.1721
Epoch 4/12
226729/226729 [==============================] - 91s 402us/sample - loss: 0.4469 - accuracy: 0.7777 - AUC: 0.8688 - MeanSquaredError: 0.1478
Epoch 5/12
226729/226729 [==============================] - 90s 397us/sample - loss: 0.3954 - accuracy: 0.8104 - AUC: 0.8999 - MeanSquaredError: 0.1288
Epoch 6/12
226729/226729 [==============================] - 100s 443us/sample - loss: 0.3514 - accuracy: 0.8366 - AUC: 0.9222 - MeanSquaredError: 0.1130
Epoch 7/12
226729/226729 [==============================] - 104

# Modelo 2 - Simple GRU (Cross Validation K = 5)

In [59]:
kfold = KFold(n_splits=5, shuffle=True)
results = cross_validate(estimator_GRU, X,dummy_y , cv=kfold)

Train on 181383 samples
Epoch 1/12
181383/181383 [==============================] - 91s 500us/sample - loss: 0.6783 - accuracy: 0.5626 - AUC: 0.5917 - MeanSquaredError: 0.2429
Epoch 2/12
181383/181383 [==============================] - 85s 471us/sample - loss: 0.6258 - accuracy: 0.6389 - AUC: 0.6976 - MeanSquaredError: 0.2191
Epoch 3/12
181383/181383 [==============================] - 88s 485us/sample - loss: 0.5663 - accuracy: 0.6920 - AUC: 0.7696 - MeanSquaredError: 0.1944
Epoch 4/12
181383/181383 [==============================] - 87s 478us/sample - loss: 0.5055 - accuracy: 0.7393 - AUC: 0.8257 - MeanSquaredError: 0.1702
Epoch 5/12
181383/181383 [==============================] - 88s 483us/sample - loss: 0.4503 - accuracy: 0.7777 - AUC: 0.8667 - MeanSquaredError: 0.1490
Epoch 6/12
181383/181383 [==============================] - 90s 495us/sample - loss: 0.4029 - accuracy: 0.8068 - AUC: 0.8958 - MeanSquaredError: 0.1315
Epoch 7/12
181383/181383 [==============================] - 87s 

In [60]:
estimator_GRU.model.save("modelo2_GRU.h5")

# Modelo 3 - Simple RNN

In [9]:
#Modelo 3 - Simple RNN 
def rnn_model():
  model = Sequential()
  model.add(SimpleRNN(64, input_shape=(X.shape[1], X.shape[2])))
  model.add(Dense(2, activation='softmax'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', 'AUC','MeanSquaredError'])
  return model

estimator_RNN = KerasClassifier(build_fn=rnn_model, epochs=12, batch_size=16) #Epochs = 12
estimator_RNN.fit(X,dummy_y)

Train on 226729 samples
Epoch 1/12
226729/226729 [==============================] - 134s 593us/sample - loss: 0.6857 - accuracy: 0.5467 - AUC: 0.5686 - MeanSquaredError: 0.2464
Epoch 2/12
226729/226729 [==============================] - 132s 583us/sample - loss: 0.6562 - accuracy: 0.6035 - AUC: 0.6477 - MeanSquaredError: 0.2325
Epoch 3/12
226729/226729 [==============================] - 129s 570us/sample - loss: 0.6124 - accuracy: 0.6552 - AUC: 0.7182 - MeanSquaredError: 0.2131
Epoch 4/12
226729/226729 [==============================] - 128s 563us/sample - loss: 0.5761 - accuracy: 0.6871 - AUC: 0.7617 - MeanSquaredError: 0.1978
Epoch 5/12
226729/226729 [==============================] - 128s 564us/sample - loss: 0.5459 - accuracy: 0.7121 - AUC: 0.7921 - MeanSquaredError: 0.1855
Epoch 6/12
226729/226729 [==============================] - 129s 569us/sample - loss: 0.5185 - accuracy: 0.7332 - AUC: 0.8172 - MeanSquaredError: 0.1744
Epoch 7/12
226729/226729 [==============================] 

# Modelo 3 - Simple RNN (Cross Validation K = 5)

In [11]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import *

kfold = KFold(n_splits=5, shuffle=True)
results = cross_validate(estimator_RNN, X,dummy_y , cv=kfold)

Train on 181383 samples
Epoch 1/12
181383/181383 [==============================] - 106s 583us/sample - loss: 0.6875 - accuracy: 0.5433 - AUC: 0.5615 - MeanSquaredError: 0.2472
Epoch 2/12
181383/181383 [==============================] - 105s 576us/sample - loss: 0.6650 - accuracy: 0.5897 - AUC: 0.6289 - MeanSquaredError: 0.2365
Epoch 3/12
181383/181383 [==============================] - 104s 573us/sample - loss: 0.6340 - accuracy: 0.6323 - AUC: 0.6873 - MeanSquaredError: 0.2225
Epoch 4/12
181383/181383 [==============================] - 104s 576us/sample - loss: 0.6076 - accuracy: 0.6609 - AUC: 0.7254 - MeanSquaredError: 0.2109
Epoch 5/12
181383/181383 [==============================] - 104s 574us/sample - loss: 0.5856 - accuracy: 0.6821 - AUC: 0.7518 - MeanSquaredError: 0.2016
Epoch 6/12
181383/181383 [==============================] - 104s 575us/sample - loss: 0.5635 - accuracy: 0.6999 - AUC: 0.7757 - MeanSquaredError: 0.1925
Epoch 7/12
181383/181383 [==============================] 

In [12]:
estimator_RNN.model.save("modelo3_RNN.h5")

# Modelo 4 - Bidirectional LSTM

In [14]:
def bi_lstm_model():
  model = Sequential()
  model.add(Bidirectional(LSTM(64, input_shape=(X.shape[1], X.shape[2]))))
  model.add(Dense(2, activation='softmax'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', 'AUC','MeanSquaredError'])
  return model

estimator_bi_lstm = KerasClassifier(build_fn=bi_lstm_model, epochs=12, batch_size=16) #Epochs = 12
estimator_bi_lstm.fit(X,dummy_y)

Train on 226729 samples
Epoch 1/12
226729/226729 [==============================] - 123s 544us/sample - loss: 0.6717 - accuracy: 0.5725 - AUC: 0.6077 - MeanSquaredError: 0.2398
Epoch 2/12
226729/226729 [==============================] - 114s 502us/sample - loss: 0.6129 - accuracy: 0.6505 - AUC: 0.7144 - MeanSquaredError: 0.2137
Epoch 3/12
226729/226729 [==============================] - 118s 519us/sample - loss: 0.5501 - accuracy: 0.7059 - AUC: 0.7857 - MeanSquaredError: 0.1879
Epoch 4/12
226729/226729 [==============================] - 116s 512us/sample - loss: 0.4939 - accuracy: 0.7475 - AUC: 0.8351 - MeanSquaredError: 0.1657
Epoch 5/12
226729/226729 [==============================] - 117s 518us/sample - loss: 0.4437 - accuracy: 0.7805 - AUC: 0.8706 - MeanSquaredError: 0.1468
Epoch 6/12
226729/226729 [==============================] - 117s 516us/sample - loss: 0.3976 - accuracy: 0.8091 - AUC: 0.8985 - MeanSquaredError: 0.1298
Epoch 7/12
226729/226729 [==============================] 

# Modelo 4 - Bidirectional LSTM (Cross Validation K = 5)

In [15]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import *

kfold = KFold(n_splits=5, shuffle=True)
results = cross_validate(estimator_bi_lstm, X,dummy_y , cv=kfold)

Train on 181383 samples
Epoch 1/12
181383/181383 [==============================] - 101s 557us/sample - loss: 0.6790 - accuracy: 0.5602 - AUC: 0.5880 - MeanSquaredError: 0.2432
Epoch 2/12
181383/181383 [==============================] - 98s 543us/sample - loss: 0.6343 - accuracy: 0.6279 - AUC: 0.6842 - MeanSquaredError: 0.2229
Epoch 3/12
181383/181383 [==============================] - 96s 527us/sample - loss: 0.5842 - accuracy: 0.6785 - AUC: 0.7505 - MeanSquaredError: 0.2016
Epoch 4/12
181383/181383 [==============================] - 98s 539us/sample - loss: 0.5321 - accuracy: 0.7202 - AUC: 0.8031 - MeanSquaredError: 0.1806
Epoch 5/12
181383/181383 [==============================] - 97s 535us/sample - loss: 0.4851 - accuracy: 0.7529 - AUC: 0.8416 - MeanSquaredError: 0.1624
Epoch 6/12
181383/181383 [==============================] - 96s 527us/sample - loss: 0.4395 - accuracy: 0.7834 - AUC: 0.8736 - MeanSquaredError: 0.1451
Epoch 7/12
181383/181383 [==============================] - 102

In [16]:
estimator_bi_lstm.model.save("modelo4_Bi_LSTM.h5")

# Modelo 5 - Bidirectional GRU

In [17]:
#Modelo 5 - GRU 
def bi_gru_model():
  model = Sequential()
  model.add(Bidirectional(GRU(64, input_shape=(X.shape[1], X.shape[2]))))
  model.add(Dense(2, activation='softmax'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', 'AUC','MeanSquaredError'])
  return model

estimator_bi_GRU = KerasClassifier(build_fn=bi_gru_model, epochs=12, batch_size=16) #Epochs = 12
estimator_bi_GRU.fit(X,dummy_y)

Train on 226729 samples
Epoch 1/12
226729/226729 [==============================] - 121s 533us/sample - loss: 0.6679 - accuracy: 0.5790 - AUC: 0.6172 - MeanSquaredError: 0.2381
Epoch 2/12
226729/226729 [==============================] - 117s 514us/sample - loss: 0.5865 - accuracy: 0.6745 - AUC: 0.7470 - MeanSquaredError: 0.2027- loss: 0.5871 - accuracy: 0.6738 - 
Epoch 3/12
226729/226729 [==============================] - 117s 516us/sample - loss: 0.5088 - accuracy: 0.7369 - AUC: 0.8229 - MeanSquaredError: 0.1715
Epoch 4/12
226729/226729 [==============================] - 115s 507us/sample - loss: 0.4433 - accuracy: 0.7817 - AUC: 0.8712 - MeanSquaredError: 0.1464
Epoch 5/12
226729/226729 [==============================] - 118s 522us/sample - loss: 0.3874 - accuracy: 0.8148 - AUC: 0.9041 - MeanSquaredError: 0.1260
Epoch 6/12
226729/226729 [==============================] - 120s 529us/sample - loss: 0.3450 - accuracy: 0.8395 - AUC: 0.9251 - MeanSquaredError: 0.1109
Epoch 7/12
226729/2267

# Modelo 5 - Bidirectional GRU (Cross Validation K = 5)

In [18]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import *

kfold = KFold(n_splits=5, shuffle=True)
results = cross_validate(estimator_bi_GRU, X,dummy_y , cv=kfold)

Train on 181383 samples
Epoch 1/12
181383/181383 [==============================] - 99s 547us/sample - loss: 0.6772 - accuracy: 0.5659 - AUC: 0.5958 - MeanSquaredError: 0.2423
Epoch 2/12
181383/181383 [==============================] - 97s 532us/sample - loss: 0.6200 - accuracy: 0.6434 - AUC: 0.7054 - MeanSquaredError: 0.2166
Epoch 3/12
181383/181383 [==============================] - 97s 534us/sample - loss: 0.5504 - accuracy: 0.7058 - AUC: 0.7860 - MeanSquaredError: 0.1878
Epoch 4/12
181383/181383 [==============================] - 98s 538us/sample - loss: 0.4876 - accuracy: 0.7523 - AUC: 0.8404 - MeanSquaredError: 0.1630
Epoch 5/12
181383/181383 [==============================] - 97s 535us/sample - loss: 0.4291 - accuracy: 0.7912 - AUC: 0.8805 - MeanSquaredError: 0.1410
Epoch 6/12
181383/181383 [==============================] - 96s 528us/sample - loss: 0.3816 - accuracy: 0.8199 - AUC: 0.9075 - MeanSquaredError: 0.1236
Epoch 7/12
181383/181383 [==============================] - 95s 

In [19]:
estimator_bi_GRU.model.save("modelo5_Bi_GRU.h5")

# Modelo 6 - Bidirectional Simple RNN

In [20]:
#Modelo 6 - Simple RNN 
def bi_rnn_model():
  model = Sequential()
  model.add(Bidirectional(SimpleRNN(64, input_shape=(X.shape[1], X.shape[2]))))
  model.add(Dense(2, activation='softmax'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', 'AUC','MeanSquaredError'])
  return model

estimator_bi_RNN = KerasClassifier(build_fn=bi_rnn_model, epochs=12, batch_size=16) #Epochs = 12
estimator_bi_RNN.fit(X,dummy_y)

Train on 226729 samples
Epoch 1/12
226729/226729 [==============================] - 289s 1ms/sample - loss: 0.6857 - accuracy: 0.5501 - AUC: 0.5721 - MeanSquaredError: 0.2464
Epoch 2/12
226729/226729 [==============================] - 283s 1ms/sample - loss: 0.6486 - accuracy: 0.6139 - AUC: 0.6624 - MeanSquaredError: 0.2291s - loss: 0.6494 - accuracy: 0.6130 -  - ETA: 7s - loss: 0.6492 - accuracy: 0.6 - ETA:
Epoch 3/12
226729/226729 [==============================] - 278s 1ms/sample - loss: 0.6048 - accuracy: 0.6631 - AUC: 0.7279 - MeanSquaredError: 0.2100
Epoch 4/12
226729/226729 [==============================] - 275s 1ms/sample - loss: 0.5666 - accuracy: 0.6961 - AUC: 0.7718 - MeanSquaredError: 0.1940
Epoch 5/12
226729/226729 [==============================] - 275s 1ms/sample - loss: 0.5349 - accuracy: 0.7218 - AUC: 0.8027 - MeanSquaredError: 0.1811s - loss: 0.5350 - accuracy: 0.7218 - AUC: 0.8026 - MeanSquaredErr - ETA: 2s - loss: 0.5348 - accuracy:
Epoch 6/12
226729/226729 [======

# Modelo 6 - Bidirectional Simple RNN (Cross Validation K = 5)

In [21]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import *

kfold = KFold(n_splits=5, shuffle=True)
results = cross_validate(estimator_bi_RNN, X,dummy_y , cv=kfold)

Train on 181383 samples
Epoch 1/12
181383/181383 [==============================] - 225s 1ms/sample - loss: 0.6876 - accuracy: 0.5446 - AUC: 0.5659 - MeanSquaredError: 0.2473s - loss: 0.6878 - accuracy: 0.5440 - AUC: 0.5651 - Mea - ETA: 3s - loss: 
Epoch 2/12
181383/181383 [==============================] - 211s 1ms/sample - loss: 0.6570 - accuracy: 0.6014 - AUC: 0.6464 - MeanSquaredError: 0.2329
Epoch 3/12
181383/181383 [==============================] - 217s 1ms/sample - loss: 0.6213 - accuracy: 0.6453 - AUC: 0.7056 - MeanSquaredError: 0.2171
Epoch 4/12
181383/181383 [==============================] - 211s 1ms/sample - loss: 0.5860 - accuracy: 0.6789 - AUC: 0.7504 - MeanSquaredError: 0.2021
Epoch 5/12
181383/181383 [==============================] - 208s 1ms/sample - loss: 0.5573 - accuracy: 0.7026 - AUC: 0.7810 - MeanSquaredError: 0.1902s - loss: 0.5575 - accuracy: 0.7025 - AUC: 0.7809 - MeanSqu - ETA: 4s - loss: 0.5576 - accuracy: 0.7025 - AUC: 0.7808 - MeanSquaredErro - ETA: 4s - 

In [22]:
estimator_bi_RNN.model.save("modelo6_Bi_RNN.h5")

# Modelo 7 - Stacked LSTM

In [8]:
#Modelo 7 - stacked LSTM 
def stacked_lstm_model():
  model = Sequential()
  model.add(LSTM(64, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
  model.add(LSTM(64))
  model.add(Dense(2, activation='softmax'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', 'AUC','MeanSquaredError'])
  return model

estimator_stacked_lstm = KerasClassifier(build_fn=stacked_lstm_model, epochs=12, batch_size=16) #Epochs = 12
estimator_stacked_lstm.fit(X,dummy_y)

Train on 226729 samples
Epoch 1/12
226729/226729 [==============================] - 110s 487us/sample - loss: 0.6739 - accuracy: 0.5679 - AUC: 0.6003 - MeanSquaredError: 0.2408
Epoch 2/12
226729/226729 [==============================] - 103s 454us/sample - loss: 0.6158 - accuracy: 0.6477 - AUC: 0.7105 - MeanSquaredError: 0.2150
Epoch 3/12
226729/226729 [==============================] - 105s 464us/sample - loss: 0.5470 - accuracy: 0.7069 - AUC: 0.7880 - MeanSquaredError: 0.1869
Epoch 4/12
226729/226729 [==============================] - 111s 490us/sample - loss: 0.4812 - accuracy: 0.7542 - AUC: 0.8436 - MeanSquaredError: 0.1613
Epoch 5/12
226729/226729 [==============================] - 104s 460us/sample - loss: 0.4234 - accuracy: 0.7906 - AUC: 0.8826 - MeanSquaredError: 0.1398
Epoch 6/12
226729/226729 [==============================] - 105s 463us/sample - loss: 0.3705 - accuracy: 0.8233 - AUC: 0.9125 - MeanSquaredError: 0.1203
Epoch 7/12
226729/226729 [==============================] 

# Modelo 7 - Stacked LSTM (Cross Validation K = 5)

In [9]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import *

kfold = KFold(n_splits=5, shuffle=True)
results = cross_validate(estimator_stacked_lstm, X,dummy_y , cv=kfold)

Train on 181383 samples
Epoch 1/12
181383/181383 [==============================] - 84s 466us/sample - loss: 0.6796 - accuracy: 0.5603 - AUC: 0.5851 - MeanSquaredError: 0.2435
Epoch 2/12
181383/181383 [==============================] - 80s 442us/sample - loss: 0.6369 - accuracy: 0.6241 - AUC: 0.6799 - MeanSquaredError: 0.2240
Epoch 3/12
181383/181383 [==============================] - 80s 442us/sample - loss: 0.5847 - accuracy: 0.6762 - AUC: 0.7494 - MeanSquaredError: 0.2020
Epoch 4/12
181383/181383 [==============================] - 80s 442us/sample - loss: 0.5381 - accuracy: 0.7138 - AUC: 0.7967 - MeanSquaredError: 0.1832
Epoch 5/12
181383/181383 [==============================] - 80s 442us/sample - loss: 0.4989 - accuracy: 0.7419 - AUC: 0.8301 - MeanSquaredError: 0.1680
Epoch 6/12
181383/181383 [==============================] - 82s 453us/sample - loss: 0.4557 - accuracy: 0.7702 - AUC: 0.8615 - MeanSquaredError: 0.1519
Epoch 7/12
181383/181383 [==============================] - 82s 

In [10]:
estimator_stacked_lstm.model.save("modelo7_stacked_LSTM.h5")

# Modelo 8 - Stacked GRU

In [11]:
#Modelo 8 - stacked GRU
def stacked_gru_model():
  model = Sequential()
  model.add(GRU(64, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
  model.add(GRU(64))
  model.add(Dense(2, activation='softmax'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', 'AUC','MeanSquaredError'])
  return model

estimator_stacked_GRU = KerasClassifier(build_fn=stacked_gru_model, epochs=12, batch_size=16) #Epochs = 12
estimator_stacked_GRU.fit(X,dummy_y)

Train on 226729 samples
Epoch 1/12
226729/226729 [==============================] - 116s 513us/sample - loss: 0.6684 - accuracy: 0.5775 - AUC: 0.6155 - MeanSquaredError: 0.2383
Epoch 2/12
226729/226729 [==============================] - 114s 503us/sample - loss: 0.5931 - accuracy: 0.6686 - AUC: 0.7392 - MeanSquaredError: 0.2055
Epoch 3/12
226729/226729 [==============================] - 115s 509us/sample - loss: 0.5113 - accuracy: 0.7342 - AUC: 0.8206 - MeanSquaredError: 0.1726
Epoch 4/12
226729/226729 [==============================] - 115s 506us/sample - loss: 0.4239 - accuracy: 0.7929 - AUC: 0.8830 - MeanSquaredError: 0.1395
Epoch 5/12
226729/226729 [==============================] - 116s 511us/sample - loss: 0.3479 - accuracy: 0.8387 - AUC: 0.9239 - MeanSquaredError: 0.1118
Epoch 6/12
226729/226729 [==============================] - 114s 505us/sample - loss: 0.2874 - accuracy: 0.8719 - AUC: 0.9490 - MeanSquaredError: 0.0905
Epoch 7/12
226729/226729 [==============================] 

# Modelo 8 - Stacked GRU (Cross Validation K = 5)

In [12]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import *

kfold = KFold(n_splits=5, shuffle=True)
results = cross_validate(estimator_stacked_GRU, X,dummy_y , cv=kfold)

Train on 181383 samples
Epoch 1/12
181383/181383 [==============================] - 92s 508us/sample - loss: 0.6738 - accuracy: 0.5702 - AUC: 0.6028 - MeanSquaredError: 0.2407
Epoch 2/12
181383/181383 [==============================] - 90s 496us/sample - loss: 0.6189 - accuracy: 0.6456 - AUC: 0.7069 - MeanSquaredError: 0.2162
Epoch 3/12
181383/181383 [==============================] - 93s 511us/sample - loss: 0.5590 - accuracy: 0.6970 - AUC: 0.7762 - MeanSquaredError: 0.1916
Epoch 4/12
181383/181383 [==============================] - 91s 500us/sample - loss: 0.4989 - accuracy: 0.7421 - AUC: 0.8304 - MeanSquaredError: 0.1679
Epoch 5/12
181383/181383 [==============================] - 91s 502us/sample - loss: 0.4296 - accuracy: 0.7888 - AUC: 0.8796 - MeanSquaredError: 0.1416
Epoch 6/12
181383/181383 [==============================] - 91s 501us/sample - loss: 0.3707 - accuracy: 0.8257 - AUC: 0.9127 - MeanSquaredError: 0.1201
Epoch 7/12
181383/181383 [==============================] - 91s 

Epoch 12/12
181383/181383 [==============================] - 63s 348us/sample - loss: 0.1230 - accuracy: 0.9528 - AUC: 0.9912 - MeanSquaredError: 0.0355
Train on 181384 samples
Epoch 1/12
181384/181384 [==============================] - 106s 586us/sample - loss: 0.6747 - accuracy: 0.5670 - AUC: 0.5988 - MeanSquaredError: 0.2412
Epoch 2/12
181384/181384 [==============================] - 102s 564us/sample - loss: 0.6196 - accuracy: 0.6446 - AUC: 0.7057 - MeanSquaredError: 0.2165
Epoch 3/12
181384/181384 [==============================] - 101s 557us/sample - loss: 0.5540 - accuracy: 0.7006 - AUC: 0.7810 - MeanSquaredError: 0.1897
Epoch 4/12
181384/181384 [==============================] - 103s 569us/sample - loss: 0.4864 - accuracy: 0.7505 - AUC: 0.8400 - MeanSquaredError: 0.1631
Epoch 5/12
181384/181384 [==============================] - 104s 573us/sample - loss: 0.4196 - accuracy: 0.7934 - AUC: 0.8851 - MeanSquaredError: 0.1383
Epoch 6/12
181384/181384 [==============================] 

In [13]:
estimator_stacked_GRU.model.save("modelo8_stacked_GRU.h5")

# Modelo 9 - Stacked RNN 

In [14]:
#Modelo 9 - STACKED Simple RNN 
def stacked_rnn_model():
  model = Sequential()
  model.add(SimpleRNN(64, input_shape=(X.shape[1], X.shape[2]), return_sequences = True))
  model.add(SimpleRNN(64))
  model.add(Dense(2, activation='softmax'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', 'AUC','MeanSquaredError'])
  return model

estimator_stacked_RNN = KerasClassifier(build_fn=stacked_rnn_model, epochs=12, batch_size=16) #Epochs = 12
estimator_stacked_RNN.fit(X,dummy_y)

Train on 226729 samples
Epoch 1/12
226729/226729 [==============================] - 255s 1ms/sample - loss: 0.6903 - accuracy: 0.5360 - AUC: 0.5486 - MeanSquaredError: 0.2486s - loss: 0.6904 - accuracy: 0. - ETA: 3s - loss
Epoch 2/12
226729/226729 [==============================] - 253s 1ms/sample - loss: 0.6825 - accuracy: 0.5561 - AUC: 0.5807 - MeanSquaredError: 0.2448
Epoch 3/12
226729/226729 [==============================] - 251s 1ms/sample - loss: 0.6704 - accuracy: 0.5829 - AUC: 0.6187 - MeanSquaredError: 0.2389
Epoch 4/12
226729/226729 [==============================] - 254s 1ms/sample - loss: 0.6542 - accuracy: 0.6093 - AUC: 0.6544 - MeanSquaredError: 0.2314
Epoch 5/12
226729/226729 [==============================] - 254s 1ms/sample - loss: 0.6541 - accuracy: 0.6134 - AUC: 0.6576 - MeanSquaredError: 0.2311
Epoch 6/12
226729/226729 [==============================] - 252s 1ms/sample - loss: 0.6351 - accuracy: 0.6364 - AUC: 0.6903 - MeanSquaredError: 0.2224
Epoch 7/12
226729/2267

# Modelo 9 - Stacked RNN (Cross Validation K = 5)

In [15]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import *

kfold = KFold(n_splits=5, shuffle=True)
results = cross_validate(estimator_stacked_RNN, X,dummy_y , cv=kfold)

Train on 181383 samples
Epoch 1/12
181383/181383 [==============================] - 215s 1ms/sample - loss: 0.6910 - accuracy: 0.5318 - AUC: 0.5454 - MeanSquaredError: 0.2489
Epoch 2/12
181383/181383 [==============================] - 211s 1ms/sample - loss: 0.6862 - accuracy: 0.5459 - AUC: 0.5638 - MeanSquaredError: 0.2467
Epoch 3/12
181383/181383 [==============================] - 208s 1ms/sample - loss: 0.6734 - accuracy: 0.5750 - AUC: 0.6076 - MeanSquaredError: 0.2405
Epoch 4/12
181383/181383 [==============================] - 209s 1ms/sample - loss: 0.6616 - accuracy: 0.5972 - AUC: 0.6370 - MeanSquaredError: 0.2350
Epoch 5/12
181383/181383 [==============================] - 309s 2ms/sample - loss: 0.6445 - accuracy: 0.6209 - AUC: 0.6711 - MeanSquaredError: 0.2271
Epoch 6/12
181383/181383 [==============================] - 259s 1ms/sample - loss: 0.6246 - accuracy: 0.6434 - AUC: 0.7017 - MeanSquaredError: 0.2183s - loss: 0.6252 - accuracy: 0.6426 - ETA:  - ETA: 0s - loss: 0.6247 - 

In [16]:
estimator_stacked_RNN.model.save("modelo9_stacked_RNN.h5")

# End of training and validation of 9 models